In [ ]:
from benchmark_consts import get_problems, get_args_and_problems, print_, PATH_FORM_HYPERPARAMS


import os
import pickle
import traceback
import argparse
import random
import math
import copy
import time
from kmodes.kprototypes import KPrototypes

import sys
sys.path.append('..')

from lib.algorithms import PathFormulation
from lib.problem import Problem
from lib.algorithms.abstract_formulation import Objective
from lib.graph_utils import compute_in_or_out_flow, path_to_edge_list, assert_flow_conservation, check_feasibility
from collections import defaultdict
import ncflow

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from partition_entities import split_generic, check_dims

TOP_DIR = 'path-form-logs'
HEADERS = [
    'problem', 'num_nodes', 'num_edges', 'traffic_seed', 'scale_factor',
    'tm_model', 'num_commodities', 'total_demand', 'algo', 'num_paths',
    'edge_disjoint', 'dist_metric', 'total_flow', 'runtime'
]
PLACEHOLDER = ','.join('{}' for _ in HEADERS)


In [ ]:
def create_edges_onehot_dict(problem, pf_original):
    paths_dict = pf_original.get_paths(problem)
    com_list = problem.commodity_list
    num_entities = len(com_list)
    num_edges = len(problem.G.edges)
    enum_edges_dict = {}
    for i, edge in enumerate(problem.G.edges):
        enum_edges_dict[edge] = i

    # create dictionary of all edges used by each commodity
    com_path_edges_dict = defaultdict(list)
    min_demand = np.inf
    max_demand = 0
    for k, (source, target, demand) in com_list:
        paths_array = paths_dict[(source, target)]
        if min_demand > demand:
            min_demand = demand
        if max_demand < demand:
            max_demand = demand
            
        for path in paths_array:
            com_path_edges_dict[(k, source, target, demand)] += list(path_to_edge_list(path))

    com_path_edges_onehot_dict = defaultdict(list)
    np_data = np.zeros((num_entities,num_edges+1))
    for (k, source, target, demand), edge_list in com_path_edges_dict.items():
        onehot_edge = [0]*num_edges
        for edge in edge_list:
            edge_i = enum_edges_dict[edge]
            onehot_edge[edge_i] = 1
            np_data[k,edge_i] = 1
        # add in normalized demand as a dimension
        norm_demand = (demand - min_demand)/(max_demand-min_demand)
        com_path_edges_onehot_dict[(k, source, target, demand)] = onehot_edge + [norm_demand]
        np_data[k,-1] = norm_demand

    return com_path_edges_onehot_dict

def split_generic_wrapper(problem, pf, num_subproblems, verbose=False, method='means'):
    
    input_dict = create_edges_onehot_dict(problem, pf)
    
    # create subproblems, zero out commodities in traffic matrix that aren't assigned to each 
    sub_problems = [problem.copy() for _ in range(num_subproblems)]
    if num_subproblems == 1:
        return sub_problems
    
    entity_assignments_lists = split_generic(input_dict, num_subproblems, verbose=verbose, method=method)
    
    for i in range(num_subproblems):

        #zero out all commodities not assigned to subproblem i
        for ind, source, target, demand in entity_assignments_lists[i]:
            for j in range(num_subproblems):
                if i == j:
                    continue
                sub_problems[j].traffic_matrix.tm[source,target] = 0

        # split the capacity of each link
        for u,v in sub_problems[i].G.edges:
            sub_problems[i].G[u][v]['capacity'] = sub_problems[i].G[u][v]['capacity']/num_subproblems

    return sub_problems
    

In [ ]:
original_dist = [2,5]
A = [[1,2,3], [4,5,6]]
B = [2,5]
C = [3,6]
#print(calc_dist(A,B,original_dist))
#print(calc_dist(A,C,original_dist))

input_dict = {'A': [1,4,3],
              'B': [4,3,6],
              'C': [7,8,2],
              'D': [1,2,3],
              'E': [4,5,6],
              'F': [9,8,1],
              'G': [1,2,4],
              'H': [4,2,3],
              'I': [7,8,5]}
subproblem_list = split_generic(input_dict,2, verbose=True, method='means')
check_dims(subproblem_list, input_dict)


topo_fname = "../topologies/topology-zoo/Ion.graphml"#GtsCe.graphml"
#tm_fname = "../traffic-matrices/uniform/GtsCe.graphml_uniform_1475504323_64.0_0.05_traffic-matrix.pkl"
tm_fname = "../traffic-matrices/uniform/Ion.graphml_uniform_1545787193_64.0_0.15_traffic-matrix.pkl"
num_paths, edge_disjoint, dist_metric = PATH_FORM_HYPERPARAMS
    
pf_original = PathFormulation.new_max_flow(
    num_paths,
    edge_disjoint=edge_disjoint,
    dist_metric=dist_metric)
with open('path-form.csv', 'a') as results:
    print_(','.join(HEADERS), file=results)
    
    problem = Problem.from_file(topo_fname, tm_fname)

    paths_dict = pf_original.get_paths(problem)
    com_list = problem.commodity_list
    num_entities = len(com_list)
    num_edges = len(problem.G.edges)
    enum_edges_dict = {}
    for i, edge in enumerate(problem.G.edges):
        enum_edges_dict[edge] = i

    # create dictionary of all edges used by each commodity
    com_path_edges_dict = defaultdict(list)
    min_demand = np.inf
    max_demand = 0
    for k, (source, target, demand) in com_list:
        paths_array = paths_dict[(source, target)]
        if min_demand > demand:
            min_demand = demand
        if max_demand < demand:
            max_demand = demand
            
        for path in paths_array:
            com_path_edges_dict[(k, source, target, demand)] += list(path_to_edge_list(path))

    com_path_edges_onehot_dict = defaultdict(list)
    np_data = np.zeros((num_entities,num_edges+1))
    for (k, source, target, demand), edge_list in com_path_edges_dict.items():
        onehot_edge = [0]*num_edges
        for edge in edge_list:
            edge_i = enum_edges_dict[edge]
            onehot_edge[edge_i] = 1
            np_data[k,edge_i] = 1
        # add in normalized demand as a dimension
        norm_demand = (demand - min_demand)/(max_demand-min_demand)
        com_path_edges_onehot_dict[k] = onehot_edge + [norm_demand]
        np_data[k,-1] = norm_demand
    
    #print("creating subproblem list...")
    #subproblem_list_meansplit = split_generic(com_path_edges_onehot_dict, 2, verbose=False)
    

In [ ]:
kp = KPrototypes(n_clusters=20, init='Huang', n_init=1, verbose=1, n_jobs=24, max_iter=8)
start_time = time.time()
clusters = kp.fit_predict(np_data, categorical=[ x for x in range(0,num_edges)])
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# paths_dict: key: (source, target), value: array of paths,
#             where a path is a list of sequential nodes
#             use lib.graph_utils.path_to_edge_list to get edges.
def split_problem_smartpath(problem, num_subproblems, paths_dict):
    com_list = problem.commodity_list

    # create dictionary of all edges used by each commodity
    com_path_edges_dict = defaultdict(list)
    for k, (source, target, demand) in com_list:
        paths_array = paths_dict[(source, target)]
        for path in paths_array:
            com_path_edges_dict[(k, source, target)] += list(path_to_edge_list(path))

    # for each edge, split all commodities using that edge across subproblems
    subproblem_com_indices = defaultdict(list)
    current_subproblem = 0
    for (u,v) in problem.G.edges:
        coms_on_edge = [x for x in com_path_edges_dict.keys() if (u,v) in com_path_edges_dict[x]]

        # split commodities that share path across all subproblems
        for (k, source, target) in coms_on_edge:
            subproblem_com_indices[current_subproblem] += [(k,source,target)]
            current_subproblem = (current_subproblem + 1) % num_subproblems
            # remove commodity from cosideration when processing later edges
            del com_path_edges_dict[(k, source, target)]

    # create subproblems, zero out commodities in traffic matrix that aren't assigned to each 
    sub_problems = []
    for i in range(num_subproblems):

        sub_problems.append(problem.copy())

        #zero out all commodities not assigned to subproblem i
        for k in subproblem_com_indices.keys():
            if k == i:
                continue
            zero_out_list = subproblem_com_indices[k]
            for ind, source, target in zero_out_list:
                sub_problems[-1].traffic_matrix.tm[source,target] = 0

        # split the capacity of each link
        for u,v in sub_problems[-1].G.edges:
            sub_problems[-1].G[u][v]['capacity'] = sub_problems[-1].G[u][v]['capacity']/num_subproblems

    return sub_problems

#Input: a Problem, and a list of number representing the divisions
def split_problem(problem, num_subproblems):
    sub_problems = []
    num_rows = len(problem.traffic_matrix.tm)
    rows_per_problem = math.floor(num_rows/num_subproblems)
    shuffled_indices = list(range(num_rows))

    for i in range(num_subproblems):

        sub_problems.append(problem.copy())
        for indx, j in enumerate(shuffled_indices):

            # zero out all rows except those in the corresponding block of shuffled indices
            # first, cover special case for last block
            if i == num_subproblems-1:
                if indx < i*rows_per_problem:
                    sub_problems[-1].traffic_matrix.tm[j,:] = 0

            elif (indx < i*rows_per_problem) or (indx >= (i+1)*rows_per_problem):
                sub_problems[-1].traffic_matrix.tm[j,:] = 0

        # split the capacity of each link
        for u,v in sub_problems[-1].G.edges:
            sub_problems[-1].G[u][v]['capacity'] = sub_problems[-1].G[u][v]['capacity']/num_subproblems

    return sub_problems

# assign commodities to subproblems at random
def split_random(problem, num_subproblems):
    sub_problems = [problem.copy() for _ in range(num_subproblems)]
    num_coms = len(problem.commodity_list)
    
    for k, (source, target, demand) in problem.commodity_list:
        sp_assignment = random.randint(0,num_subproblems-1)
        for sp in range(num_subproblems):
            if sp == sp_assignment:
                continue
            sub_problems[sp].traffic_matrix.tm[source,target] = 0
    for sub_problem in sub_problems:
        for u,v in sub_problems[-1].G.edges:
            sub_problem.G[u][v]['capacity'] = sub_problem.G[u][v]['capacity']/num_subproblems
    return sub_problems

In [ ]:
# Sweep topos and traffic matrices for that topo. For each combo, record the
# runtime and total flow for each algorithm
# split can be random, means, covs, clusters, tailored, or skewed
def benchmark_split(problems, num_subproblems_list, obj, split='random'):
    num_paths, edge_disjoint, dist_metric = PATH_FORM_HYPERPARAMS

    if (obj == 'max_flow'):
        pf_original = PathFormulation.new_max_flow(
            num_paths,
            edge_disjoint=edge_disjoint,
            dist_metric=dist_metric)
    elif (obj == 'min_max_link_util'):
        pf_original = PathFormulation.new_min_max_link_util(
            num_paths,
            edge_disjoint=edge_disjoint,
            dist_metric=dist_metric)
    else:
        print(obj + " not supported")
        return

    all_results = {}
    all_runtimes = {}
    all_sol_dicts = {}
    with open('path-form.csv', 'a') as results:
        print_(','.join(HEADERS), file=results)
        for problem_name, topo_fname, tm_fname in problems:
            problem = Problem.from_file(topo_fname, tm_fname)

            paths_dict = pf_original.get_paths(problem)

            problem_results = [[] for _ in range(len(num_subproblems_list))]
            problem_runtimes = [[] for _ in range(len(num_subproblems_list))]
            problem_sol_dicts = [[] for _ in range(len(num_subproblems_list))]
            for nsp_i, num_subproblems in enumerate(num_subproblems_list):
                
                if split == 'tailored':
                    problem_list = split_problem_smartpath(problem, num_subproblems, paths_dict)
                elif split == 'skewed':
                    problem_list = split_problem(problem, num_subproblems)
                elif split == 'random':
                    problem_list = split_random(problem, num_subproblems)
                elif split == 'means' or split == 'covs':
                    problem_list = split_generic_wrapper(problem, pf_original, 
                                                         num_subproblems, method=split)
              
                sum_obj_val = 0
                for sp_i, sub_problem in enumerate(problem_list):

                    print_(sub_problem.name, tm_fname)
                    traffic_seed = sub_problem.traffic_matrix.seed
                    total_demand = sub_problem.total_demand
                    print_('traffic seed: {}'.format(traffic_seed))
                    print_('traffic scale factor: {}'.format(
                        sub_problem.traffic_matrix.scale_factor))
                    print_('traffic matrix model: {}'.format(
                        sub_problem.traffic_matrix.model))
                    print_('total demand: {}'.format(total_demand))

                    run_dir = os.path.join(
                        TOP_DIR, sub_problem.name,
                        '{}-{}'.format(traffic_seed, sub_problem.traffic_matrix.model))
                    if not os.path.exists(run_dir):
                        os.makedirs(run_dir)

                    try:
                        print_(
                            '\\nPath formulation, {} paths, edge disjoint {}, dist metric {}'
                            .format(num_paths, edge_disjoint, dist_metric))
                        with open(
                                os.path.join(
                                    run_dir,
                                    '{}-path-formulation_{}-paths_edge-disjoint-{}_dist-metric-{}.txt'
                                    .format(sub_problem.name, num_paths, edge_disjoint,
                                            dist_metric)), 'w') as log:

                            if (obj == 'max_flow'):
                                pf = PathFormulation.new_max_flow(
                                    num_paths,
                                    edge_disjoint=edge_disjoint,
                                    dist_metric=dist_metric,
                                    out=log)
                            elif (obj == 'min_max_link_util'):
                                pf = PathFormulation.new_min_max_link_util(
                                    num_paths,
                                    edge_disjoint=edge_disjoint,
                                    dist_metric=dist_metric,
                                    out=log)
                            else:
                                print(obj + " not supported")
                                break

                            pf.solve(sub_problem)
                            pf_sol_dict = pf.extract_sol_as_dict()
                            with open(
                                    os.path.join(
                                        run_dir,
                                        '{}-path-formulation_{}-paths_edge-disjoint-{}_dist-metric-{}_sol-dict.pkl'
                                        .format(sub_problem.name, num_paths, edge_disjoint,
                                                dist_metric)), 'wb') as w:
                                pickle.dump(pf_sol_dict, w)

                        result_line = PLACEHOLDER.format(
                            sub_problem.name,
                            len(sub_problem.G.nodes),
                            len(sub_problem.G.edges),
                            traffic_seed,
                            sub_problem.traffic_matrix.scale_factor,
                            sub_problem.traffic_matrix.model,
                            len(problem.commodity_list),
                            total_demand,
                            'path_formulation',
                            num_paths,
                            edge_disjoint,
                            dist_metric,
                            pf.obj_val,
                            pf.runtime,
                        )
                        print_(result_line, file=results)
                        problem_results[nsp_i].append(pf.obj_val)
                        problem_runtimes[nsp_i].append(pf.runtime)
                        problem_sol_dicts[nsp_i].append(pf_sol_dict)
                        sum_obj_val += pf.obj_val
                    except Exception:
                        print_(
                            'Path formulation {} paths, edge disjoint {}, dist metric {}, Problem {}, traffic seed {}, traffic model {} failed'
                            .format(num_paths, edge_disjoint, dist_metric,
                                    sub_problem.name, traffic_seed,
                                    sub_problem.traffic_matrix.model))
                        traceback.print_exc(file=sys.stdout)
                print("sum of obj vals: " + str(sum_obj_val))

            all_results[(problem_name, topo_fname, tm_fname)] = problem_results
            all_runtimes[(problem_name, topo_fname, tm_fname)] = problem_runtimes
            all_sol_dicts[(problem_name, topo_fname, tm_fname)] = problem_sol_dicts
    return all_results, all_runtimes, all_sol_dicts

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--dry-run',
                        dest='dry_run',
                        action='store_true',
                        default=False)

parser.add_argument('--slices',
                        type=int,
                        choices=range(5),
                        nargs='+',
                        required=True)
args = parser.parse_args("--slices 0".split())

if not os.path.exists(TOP_DIR):
    os.makedirs(TOP_DIR)

#problems = get_problems(args)

if args.dry_run:
    print('Problems to run:')
    for problem in problems:
        print(problem)

p1 = ("uniform 64", "../topologies/topology-zoo/GtsCe.graphml", 
                        "../traffic-matrices/uniform/GtsCe.graphml_uniform_1475504323_64.0_0.05_traffic-matrix.pkl")
p2 = ("poisson-inter 128", "../topologies/topology-zoo/GtsCe.graphml", 
                        "../traffic-matrices/poisson-high-inter/GtsCe.graphml_poisson_2035531455_128.0_1000.0_0.9_8.5e-05_traffic-matrix.pkl")
p3 = ("poisson-intra 128", "../topologies/topology-zoo/GtsCe.graphml", 
                        "../traffic-matrices/poisson-high-intra/GtsCe.graphml_poisson_1367969278_128.0_200000000.0_0.1_2.25e-06_traffic-matrix.pkl")
p4 = ("uniform 8", "../topologies/topology-zoo/GtsCe.graphml", 
                        "../traffic-matrices/uniform/GtsCe.graphml_uniform_19019979_8.0_0.05_traffic-matrix.pkl")

p5 = ("kdl_16", "../topologies/topology-zoo/Kdl.graphml",
         "../traffic-matrices/gravity/Kdl.graphml_gravity_1710674203_16.0_24000.064453125_True_traffic-matrix.pkl")
p6 = ("kdl_32", "../topologies/topology-zoo/Kdl.graphml",
         "../traffic-matrices/gravity/Kdl.graphml_gravity_1836337794_32.0_48001.9765625_True_traffic-matrix.pkl")
p7 = ("kdl_64", "../topologies/topology-zoo/Kdl.graphml",
         "../traffic-matrices/gravity/Kdl.graphml_gravity_530572184_64.0_95909.890625_True_traffic-matrix.pkl")
p8 = ("kdl_128", "../topologies/topology-zoo/Kdl.graphml",
         "../traffic-matrices/gravity/Kdl.graphml_gravity_378818577_128.0_192067.765625_True_traffic-matrix.pkl")

p9 = ("cogentco_16", "../topologies/topology-zoo/Cogentco.graphml",
         "../traffic-matrices/gravity/Cogentco.graphml_gravity_1443251160_16.0_19189.4609375_True_traffic-matrix.pkl")
p10 = ("cogentco_32", "../topologies/topology-zoo/Cogentco.graphml",
         "../traffic-matrices/gravity/Cogentco.graphml_gravity_1625678606_32.0_38304.9609375_True_traffic-matrix.pkl")
p11 = ("cogentco_64", "../topologies/topology-zoo/Cogentco.graphml",
         "../traffic-matrices/gravity/Cogentco.graphml_gravity_1693196370_64.0_76793.84375_True_traffic-matrix.pkl")
p12 = ("cogentco_128", "../topologies/topology-zoo/Cogentco.graphml",
         "../traffic-matrices/gravity/Cogentco.graphml_gravity_1882736072_128.0_153591.515625_True_traffic-matrix.pkl")

p13 = ("kdl_8", "../topologies/topology-zoo/Kdl.graphml",
         "../traffic-matrices/gravity/Kdl.graphml_gravity_2139423624_8.0_11990.4140625_True_traffic-matrix.pkl")

p14 = ("ER_32", "../topologies/erdos-renyi-1260231677.json",
       "../traffic-matrices/gravity/erdos-renyi-1260231677.json_gravity_479716829_32.0_15993702.0_True_traffic-matrix.pkl")

p15 = ("ion_64", "../topologies/topology-zoo/Ion.graphml",
       "../traffic-matrices/uniform/Ion.graphml_uniform_1545787193_64.0_0.15_traffic-matrix.pkl")

#num_subproblems = [64]
num_subproblems = [1,2]#,4,8,16,32]
#num_subproblems = [1,8]
#problems = [(os.path.basename(p1[1]), p1[1], p1[2])]
problems = [p15]#,p10,p11]
#problems = [p5,p6,p7,p8,p9,p10,p11,p12]

obj_types = ["max_flow"]#, "min_max_link_util"]
#obj_types = ["min_max_link_util"]"

In [ ]:
obj_type = 'max_flow'
split_methods = ['random', 'skewed', 'tailored']#, 'covs', 'skewed', 'tailored']
results_all = {}
runtimes_all = {}
sol_dicts_all = {}
for method in split_methods:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, split=method)
    results_all[method] = results
    runtimes_all[method] = runtimes
    sol_dicts_all[method] = sol_dicts

In [ ]:
colors = ['k', 'r', 'b', 'g', 'm']
linestyles = ['-', '--', ':']
#print(results_ncflow)
#results_ncflow = results_ncflow[1::2]
#print(runtimes_ncflow)
def compute_obj_val(obj, problem, obj_vals, sol_dicts):
    if obj == 'max_flow':
        return sum(obj_vals)
    elif obj == 'min_max_link_util':
        link_utils_dict = defaultdict(int)
        for sol_dict in sol_dicts:
            for flow_list in sol_dict.values():
                for ((u, v), flow_val) in flow_list: # TODO: is this right? Or is it ((u, v), flow_val)?
                    link_utils_dict[(u, v)] += flow_val
        
        # TODO: is this right? Or is it ((u, v), c_e)?
        link_utils = {(u,v): link_utils_dict[(u, v)]/c_e for (u,v,c_e) in problem.G.edges.data('capacity')}
        return max(link_utils.values())
    
# run the benchmarks, plot results
def plot_split_results(results_all, runtimes_all, sol_dicts_all, 
                  results_cspf, runtimes_cspf):
    fig_results, ax_results = plt.subplots()
    plt.xlabel('number of subproblems')
    plt.ylabel('total flow')
    
    fig_rr, ax_rr = plt.subplots()
    
    plt.xlabel('execution time (seconds)')
    plt.ylabel('total allocated flow')
    #ymin, ymax = plt.ylim()
    #plt.ylim([0,ymax*1.1])
    
    for m_i, method in enumerate(split_methods):
        
        results = results_all[method]
        runtimes = runtimes_all[method]
        sol_dicts = sol_dicts_all[method]
      
        
        for p_i, p in enumerate(problems):
            total_obj_values = []
            for n_i, n in enumerate(num_subproblems):
                total_obj_val = compute_obj_val(obj_type, Problem.from_file(p[1], p[2]),
                                                results[p][n_i], sol_dicts[p][n_i])
        #         sum_val = sum(results[p][n_i])
                total_obj_values.append(total_obj_val)
            ax_results.plot(num_subproblems, total_obj_values, marker='*', c=colors[m_i%len(colors)], 
                            linestyle=linestyles[m_i%len(linestyles)], label=method)

        # plot performance vs runtime.
        for p_i, p in enumerate(problems):
            total_obj_values = []
            total_runtime = []
            max_runtime = []
            for n_i, n in enumerate(num_subproblems):
                total_obj_val = compute_obj_val(obj_type, Problem.from_file(p[1], p[2]),
                                                results[p][n_i], sol_dicts[p][n_i])
                total_obj_values.append(total_obj_val)

                sum_val = sum(runtimes[p][n_i])
                max_time = max(runtimes[p][n_i])
                total_runtime.append(sum_val)
                max_runtime.append(max_time)
            print(max_runtime)
            print(total_obj_values)
            ax_rr.scatter(max_runtime, total_obj_values, marker='*', c=colors[m_i%len(colors)], label=method)
            for n_i, n in enumerate(num_subproblems):
                ax_rr.annotate(n, (max_runtime[n_i], total_obj_values[n_i]))
            #if (obj_type == "max_flow"):
                #also plot runtime of ncflow
            #    ax.plot(runtimes_ncflow[p_i], results_ncflow[p_i], marker='P', 
            #            markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
            ax_rr.plot(runtimes_cspf[p_i], results_cspf[p_i], marker='x', 
                    markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; CSPF")
    ax_results.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax_rr.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax_rr.set_xscale('log')    

plot_split_results(results_all, runtimes_all, 
               sol_dicts_all,
               results_cspf, runtimes_cspf)

## Firas's Work

In [ ]:
problem_name, topo_fname, tm_fname = p8
prob = Problem.from_file(topo_fname, tm_fname)
NUM_PATHS = 4
NUM_SUBPROBLEMS = 8

In [ ]:
def solve_subproblems(problem_list):
    sol_dicts, runtimes, obj_vals = [], [], []
    for sub_problem in problem_list:
        pf = PathFormulation.get_pf_for_obj(Objective.MAX_FLOW, NUM_PATHS)
        pf.solve(sub_problem)
        sol_dicts.append(pf.extract_sol_as_dict())
        runtimes.append(pf.runtime)
        obj_vals.append(pf.obj_val)
    #check
    return sol_dicts, runtimes, obj_vals

def solve_and_check_feasiblity(problem, num_subproblems, num_paths):    
    paths_dict = PathFormulation.new_max_flow(num_paths).get_paths(problem)
    problem_list = split_problem_smartpath(prob, num_subproblems, paths_dict)
    sol_dicts, runtimes, obj_vals = solve_subproblems(problem_list)
    check_feasibility(problem, sol_dicts)

In [ ]:
solve_and_check_feasiblity(prob, NUM_SUBPROBLEMS, NUM_PATHS)

# End Firas's Work

In [ ]:
def validate_solution(sol_dicts_all, num_subproblems):
    for obj_type in obj_types:
        sol_dicts = sol_dicts_all[obj_type]
        for p_spec in problems:
            problem = Problem.from_file(p_spec[1], p_spec[2a])
            com_list = problem.commodity_list
            
            for n_i, n in enumerate(num_subproblems):
                merged_sol_dict = defaultdict(int)
                for j in range(n):
                    sol_dict = sol_dicts[p_spec][n_i][j]
                    total_flow = 0
                    for commod_key, flow_list in sol_dict.items():
                        assert_flow_conservation(flow_list, commod_key)
                        src, target = commod_key[-1][0], commod_key[-1][1]

                        flow = compute_in_or_out_flow(flow_list, 0, {commod_key[-1][0]})

                        merged_sol_dict[(src,target)] += flow
            
                frac_demands_satisfied = {commod_key:
                                      merged_sol_dict[(commod_key[-1][0],
                                                       commod_key[-1][1])] / commod_key[-1][-1]
                                      for commod_key in com_list}
                for commod_key, frac in frac_demands_satisfied.items():
                    if frac > 1:
                        print("assertion error, demand oversatisfied "+ str(commod_key) + " " + str(frac))
                        break
                    

In [ ]:
results_all_obj = {}
runtimes_all_obj = {}
sol_dicts_all_obj = {}
for obj_type in obj_types:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, smart=False)
    results_all_obj[obj_type] = results
    runtimes_all_obj[obj_type] = runtimes
    sol_dicts_all_obj[obj_type] = sol_dicts

In [ ]:
results_all_obj_smart = {}
runtimes_all_obj_smart = {}
sol_dicts_all_obj_smart = {}
for obj_type in obj_types:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, smart=True, generic=False)
    results_all_obj_smart[obj_type] = results
    runtimes_all_obj_smart[obj_type] = runtimes
    sol_dicts_all_obj_smart[obj_type] = sol_dicts

In [ ]:
results_all_obj_smart_gen = {}
runtimes_all_obj_smart_gen = {}
sol_dicts_all_obj_smart_gen = {}
for obj_type in obj_types:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, smart=True, generic=True)
    results_all_obj_smart_gen[obj_type] = results
    runtimes_all_obj_smart_gen[obj_type] = runtimes
    sol_dicts_all_obj_smart_gen[obj_type] = sol_dicts

In [ ]:
smart_results = [results_all_obj_smart, runtimes_all_obj_smart, sol_dicts_all_obj_smart]
pickle.dump(smart_results, open("results/smart_results.pkl", "wb"))

In [ ]:
[results_all_obj_smart, 
runtimes_all_obj_smart, 
sol_dicts_all_obj_smart] = pickle.load(open("results/smart_results.pkl", "rb"))

In [ ]:
#run NCFlow on problem set
problems_ncflow = [(os.path.basename(p[1]), p[1], p[2]) for p in problems]
results_ncflow, runtimes_ncflow = ncflow.benchmark(problems_ncflow)

In [ ]:
colors = ['k', 'r', 'b', 'g']
linestyles = ['-', '--', ':']
#print(results_ncflow)
#results_ncflow = results_ncflow[1::2]
#print(runtimes_ncflow)
def compute_obj_val(obj, problem, obj_vals, sol_dicts):
    if obj == 'max_flow':
        return sum(obj_vals)
    elif obj == 'min_max_link_util':
        link_utils_dict = defaultdict(int)
        for sol_dict in sol_dicts:
            for flow_list in sol_dict.values():
                for ((u, v), flow_val) in flow_list: # TODO: is this right? Or is it ((u, v), flow_val)?
                    link_utils_dict[(u, v)] += flow_val
        
        # TODO: is this right? Or is it ((u, v), c_e)?
        link_utils = {(u, v): link_utils_dict[(u, v)] / c_e for (u, v, c_e) in problem.G.edges.data('capacity')}
        return max(link_utils.values())
    
# run the benchmarks, plot results
def plot_benchmark(results_all, runtimes_all, sol_dicts_all, results_ncflow, runtimes_ncflow,
                  results_cspf, runtimes_cspf):
    for obj_type in obj_types:
        
        results = results_all[obj_type]
        runtimes = runtimes_all[obj_type]
        sol_dicts = sol_dicts_all[obj_type]
        
        fig, ax = plt.subplots()
        for p_i, p in enumerate(problems):
            total_obj_values = []
            for n_i, n in enumerate(num_subproblems):
                total_obj_val = compute_obj_val(obj_type, Problem.from_file(p[1], p[2]),
                                                results[p][n_i], sol_dicts[p][n_i])
        #         sum_val = sum(results[p][n_i])
                total_obj_values.append(total_obj_val)
            ax.plot(num_subproblems, total_obj_values, marker='*', c=colors[p_i%len(colors)], linestyle=linestyles[p_i%len(linestyles)],
                    label=p[0])
            #if (obj_type == "max_flow"):
                #also plot flow achieved by ncflow
                #ax.plot(1, results_ncflow[p_i], marker='P', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlabel('number of subproblems')
        plt.ylabel('total flow')
        #plt.ylim([0,None])
        #plt.savefig('./plots/flow_'+p[0]+'.png', bbox_inches='tight')

        fig, ax = plt.subplots()
        for p_i, p in enumerate(problems):
            total_runtime = []
            max_runtime = []
            for n_i, n in enumerate(num_subproblems):
                sum_val = sum(runtimes[p][n_i])
                print(sum_val)
                max_time = max(runtimes[p][n_i])
                total_runtime.append(sum_val)
                max_runtime.append(max_time)
            ax.plot(num_subproblems, total_runtime, marker='*', linestyle=linestyles[0], 
                    c=colors[p_i%len(colors)], label='sum ' + p[0])
            ax.plot(num_subproblems, max_runtime, marker='^', linestyle=linestyles[1], 
                    c=colors[p_i%len(colors)], label='max ' + p[0])
            if (obj_type == "max_flow"):
                #also plot runtime of ncflow
                ax.plot(1, runtimes_ncflow[p_i], marker='P', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlabel('number of subproblems')
        plt.ylabel('execution time (seconds)')
        plt.ylim([0,None])
        #plt.savefig('./plots/runtime_'+p[0]+'.png', bbox_inches='tight')

        # plot performance vs runtime.
        fig, ax = plt.subplots()
        for p_i, p in enumerate(problems):
            total_obj_values = []
            total_runtime = []
            max_runtime = []
            for n_i, n in enumerate(num_subproblems):
                total_obj_val = compute_obj_val(obj_type, Problem.from_file(p[1], p[2]),
                                                results[p][n_i], sol_dicts[p][n_i])
                total_obj_values.append(total_obj_val)

                sum_val = sum(runtimes[p][n_i])
                max_time = max(runtimes[p][n_i])
                total_runtime.append(sum_val)
                max_runtime.append(max_time)
            print(max_runtime)
            print(total_obj_values)
            ax.scatter(max_runtime, total_obj_values, marker='*', c=colors[p_i%len(colors)], label=p[0])
            for n_i, n in enumerate(num_subproblems):
                ax.annotate(n, (max_runtime[n_i], total_obj_values[n_i]))
            if (obj_type == "max_flow"):
                #also plot runtime of ncflow
                ax.plot(runtimes_ncflow[p_i], results_ncflow[p_i], marker='P', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
                ax.plot(runtimes_cspf[p_i], results_cspf[p_i], marker='x', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; CSPF")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlabel('execution time (seconds)')
        plt.ylabel('total allocated flow')
        ymin, ymax = plt.ylim()
        plt.ylim([0,ymax*1.1])
        ax.set_xscale('log')
        """
        for p_spec in probs:
            break
            problem = Problem.from_file(p_spec[1], p_spec[2])
            com_list = problem.commodity_list

            fig,ax = plt.subplots()

            # plot the demand distribution
            demands = [ com_list[i][1][2] for i in range(len(com_list)) ]
            num_bins = 100
            counts, bin_edges = np.histogram(demands, bins=num_bins)
            cdf = np.cumsum(counts)
            ax.plot(bin_edges[1:], cdf/cdf[-1], c='m', label="demands")

            for n_i, n in enumerate(num_subproblems):
                merged_sol_dict = defaultdict(int)
                for j in range(n):
                    sol_dict = sol_dicts[p_spec][n_i][j]


                    total_flow = 0
                    for commod_key, flow_list in sol_dict.items():
                        src, target = commod_key[-1][0], commod_key[-1][1]

                        flow = compute_in_or_out_flow(flow_list, 0, {commod_key[-1][0]})

                        merged_sol_dict[(src,target)] += flow

                    # get amount of flow assigned to each commodity (ASSUMING SINGLE PATH)
                    #flow_counts += [ sol_dict[sdf][0][1] for sdf in sol_dict if len(sol_dict[sdf]) > 0 ]

                frac_demands_satisfied = {commod_key:
                                          merged_sol_dict[(commod_key[-1][0],
                                                           commod_key[-1][1])] / commod_key[-1][-1]
                                          for commod_key in com_list}

                # take frac_demands_satisfied, extract list, plot cdf
                num_bins = 100
                #print("sol_dict: " + str(sum(flow_counts)) + ", actual obj: " + str(sum(results[p][n_i])))
                counts, bin_edges = np.histogram(flow_counts, bins=num_bins)
                cdf = np.cumsum(counts)
                ax.plot(bin_edges[1:], cdf/cdf[-1], c=colors[n_i], label=str(n)+" subproblems")

            plt.xlabel('per-commodity allocated flow')
            plt.ylabel('cumulative')
            plt.legend()
"""
    

In [ ]:
#problems = [p6]
#plot_benchmark(results_all_obj, runtimes_all_obj, sol_dicts_all_obj, 
#               results_ncflow, runtimes_ncflow)
#results_ncflow = None
#runtimes_ncflow = None
plot_benchmark(results_all_obj_smart, runtimes_all_obj_smart, 
               sol_dicts_all_obj_smart, results_ncflow, runtimes_ncflow,
               results_cspf, runtimes_cspf)

plot_benchmark(results_all_obj_smart_gen, runtimes_all_obj_smart_gen, 
               sol_dicts_all_obj_smart_gen, results_ncflow, runtimes_ncflow,
               results_cspf, runtimes_cspf)


In [ ]:

#problem = Problem.from_file("../topologies/topology-zoo/GtsCe.graphml", 
#                            "../traffic-matrices/uniform/GtsCe.graphml_uniform_1475504323_64.0_0.05_traffic-matrix.pkl")
#problem.G
#print(problem.G.edges.data())
#com_list = problem.commodity_list
#problem2 = problem.copy()
#print(dir(problem))

#pf = PathFormulation.new_max_flow(
#                    num_paths,
#                    edge_disjoint=edge_disjoint,
#                    dist_metric=dist_metric)
        
#paths_dict = pf.get_paths(problem)
#com0 = com_list[3]
#print(com0)
#print(paths_dict[(com0[1][0],com0[1][1])])

"""
problem2.traffic_matrix.tm
new_tm = problem2.traffic_matrix.tm[0:10,:]

num_rows = len(problem2.traffic_matrix.tm)

shuffled_indices = list(range(num_rows))
random.shuffle(shuffled_indices)

num_first_problem = math.floor(num_rows/2)

for i in shuffled_indices[1:num_first_problem]:
    problem2.traffic_matrix.tm[i,:] = 0

#print(problem2.traffic_matrix.tm[1:5,:])

for u,v in problem.G.edges:
    problem.G[u][v]['capacity'] = problem.G[u][v]['capacity']/2
    problem2.G[u][v]['capacity'] = problem2.G[u][v]['capacity']/2
"""

In [ ]:
def CSPF(problems):
    
    results_all = []
    runtimes_all = []
    
    for problem_name, topo_fname, tm_fname in problems:
        problem = Problem.from_file(topo_fname, tm_fname)
        
        com_list = problem.commodity_list
        tm = problem.traffic_matrix.tm
        
        pf = PathFormulation.new_max_flow(
                    num_paths,
                    edge_disjoint=edge_disjoint,
                    dist_metric=dist_metric)
        
        paths_dict = pf.get_paths(problem)
        
        # initialize link capacity dict
        remaining_link_capacity_dict = {}
        for u,v in problem.G.edges:
            remaining_link_capacity_dict[(u,v)] = problem.G[u][v]['capacity']
            
        # sort paths in ascending order
        all_paths_list = []
        allocated_coms = {}
        for k, (source, target, demand) in com_list:
            paths_array = paths_dict[(source, target)]
            all_paths_list += paths_array
            allocated_coms[(source,target)] = False
        all_paths_list.sort(key=len)
        
        # iterate through sorted paths
        total_allocated_flow = 0
        startTime = datetime.now()
        for path in all_paths_list:
            source = path[0]
            target = path[-1]
            demand = tm[source,target]
            
            # skip if we have already allocated this commodity
            if allocated_coms[(source,target)]:
                continue
            
            # check that each edge in list has enough capacity
            edge_list = list(path_to_edge_list(path))
            room = True
            for u,v in edge_list:
                if remaining_link_capacity_dict[(u,v)] < demand:
                    room = False
                    break
            
            if not room:
                continue
            
            # allocate
            for u,v in edge_list:
                remaining_link_capacity_dict[(u,v)] -= demand
            allocated_coms[(source, target)] = True
            total_allocated_flow += demand
        runtime = datetime.now() - startTime
        
        results_all.append(total_allocated_flow)
        runtimes_all.append(runtime.total_seconds())
        print("Runtime: " + str(runtime))
        print("Allocated Flow:" + str(total_allocated_flow))
    return results_all, runtimes_all

In [ ]:
num_paths, edge_disjoint, dist_metric = PATH_FORM_HYPERPARAMS
results_cspf, runtimes_cspf = CSPF(problems)

In [ ]:
print(results_ncflow)